<a href="https://colab.research.google.com/github/ladsong/if697-2020.2-data-science/blob/projeto1/Spotify_year_trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de trends do Spotify
O objetivo desse projeto é analisar os datasets do Spotify, [disponível no Kaggle](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks), para extrair informações como quais os gêneros mais escutados ao longo dos anos e suas caracteristicas musicais.

Dos datasets fornecidos, iremos utilizar o `data_o.csv` e o `tracks.csv`, cujo já detém os dados que precisamos, como genero, ano, popularidade, [features do spotify](https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-audio-features), etc.


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Coleta
Nesse primeiro momento iremos carregar o dataset do Kaggle em nosso notebook e analisar os dados enviados, para dá inicio ao pré processamento.

### Kaggle

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle é o caminho onde o arquivo kaggle.json está presente do Google Drive
#Mudar o diretorio
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [4]:
!kaggle datasets download -d yamaerenay/spotify-dataset-19212020-160k-tracks

 98% 191M/195M [00:02<00:00, 71.7MB/s]
100% 195M/195M [00:02<00:00, 81.6MB/s]


In [5]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

Archive:  spotify-dataset-19212020-160k-tracks.zip
replace artists.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: artists.csv             
  inflating: data_by_artist_o.csv    
  inflating: data_by_genres_o.csv    
  inflating: data_by_year_o.csv      
  inflating: data_o.csv              
  inflating: dict_artists.json       
  inflating: tracks.csv              


### Carregando o dataset

In [6]:
df_data = pd.read_csv('data_o.csv')
df_tracks = pd.read_csv('tracks.csv')
df_data_by_genres = pd.read_csv('data_by_genres_o.csv')
df_data_by_artist = pd.read_csv('data_by_artist_o.csv')
df_data_by_year = pd.read_csv('data_by_year_o.csv')

In [7]:
print(f"DF - df_data:")
display(df_data.head())


DF - df_data:


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [8]:
print(f"# DF - df_tracks:")
display(df_tracks.head())

# DF - df_tracks:


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [9]:
print(f"# DF - df_data_by_artist:")
display(df_data_by_artist.head())

# DF - df_data_by_artist:


,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,['show tunes'],"""Cats"" 1981 Original London Cast",0.590111,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5,1,9
1,[],"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,30.576923,5,1,26
2,[],"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.857143,0,1,7
3,[],"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.851852,0,1,27
4,[],"""Joseph And The Amazing Technicolor Dreamcoat""...",0.510714,0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,43.000000,5,1,7


In [10]:
print(f"# DF - df_data_by_year:")
display(df_data_by_year.head())

# DF - df_data_by_year:


,mode,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,1921,0.886896,0.418597,260537.166667,0.231815,0.344878,0.205710,-17.048667,0.073662,101.531493,0.379327,0.653333,2
1,1,1922,0.938592,0.482042,165469.746479,0.237815,0.434195,0.240720,-19.275282,0.116655,100.884521,0.535549,0.140845,10
2,1,1923,0.957247,0.577341,177942.362162,0.262406,0.371733,0.227462,-14.129211,0.093949,114.010730,0.625492,5.389189,0
3,1,1924,0.940200,0.549894,191046.707627,0.344347,0.581701,0.235219,-14.231343,0.092089,120.689572,0.663725,0.661017,10
4,1,1925,0.962607,0.573863,184986.924460,0.278594,0.418297,0.237668,-14.146414,0.111918,115.521921,0.621929,2.604317,5


In [11]:
print(f"# DF - df_data_by_genre:")
display(df_data_by_genres.head())

# DF - df_data_by_genre:


,mode,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.361600,-31.514333,0.040567,75.336500,0.103783,27.833333,6
1,1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.131000,-16.854000,0.076817,120.285667,0.221750,52.500000,5
2,1,8-bit,0.762000,0.712000,1.151770e+05,0.818000,0.876000,0.126000,-9.180000,0.047000,133.444000,0.975000,48.000000,7
3,1,[],0.651417,0.529093,2.328809e+05,0.419146,0.205309,0.218696,-12.288965,0.107872,112.857352,0.513604,20.859882,7
4,1,a cappella,0.676557,0.538961,1.906285e+05,0.316434,0.003003,0.172254,-12.479387,0.082851,112.110362,0.448249,45.820071,7


# 2. Pré Processamento

Analisar dados que não façam sentindo
Explicar o que é tempo e pq retirar as linhas com o valor igual a zero

In [ ]:
df_data.loc[df_data['tempo'] == 0]

In [ ]:
df_data.loc[df_data['duration_ms'] == 0]

In [ ]:
df_data.loc[df_data['year'] == 0]

## Análise de correlações


In [ ]:
corr = df_data.corr(method='pearson')
plt.figure(figsize=(14,8))
sns.heatmap(corr, annot= True)
plt.title('Correlação')
plt.show()

Podemos observar alguns campos com correção muito baixa com o campo popularidade, como valence, explicit, key, liveness, loudness, mode. Com isso, vamos retirar essas colunas

In [ ]:
df_data.drop(['valence', 'explicit', 'key', 'liveness', 'loudness', 'mode'], axis = 1, inplace=True )

#### Nova correlação ####
corr = df_data.corr(method='pearson')
plt.figure(figsize=(14,8))
sns.heatmap(corr, annot= True)
plt.title('Correlação')
plt.show()

## Adicionando dataset de gênero
Dado que o dataset `df_data` não possue os gêneros das músicas e o nosso objetivo é analisar a popularidade dos generos com o passar dos anos, iremos adicionar o dataset `data_by_genres`

Verificar se existem generos vazios


In [ ]:
df_data_by_artist.loc[df_data_by_artist['genres'].notna()]

In [ ]:
df = pd.merge(df_data, df_tracks, left_on='id', right_on='id', how='inner', suffixes=('_data','_by_artist'))

In [ ]:
display(df)